In [1]:
from Utility import *
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

from sklearn.impute import SimpleImputer

giocatori = pd.read_csv(os.path.realpath('..\dataset\giocatori.csv'))

giocatori = giocatori.drop('nome_giocatore', axis = 1)
giocatori = giocatori.drop('numero_giocatore', axis = 1)
giocatori = giocatori.drop('squadra', axis = 1)
giocatori = giocatori.drop('RIM_D', axis = 1)
giocatori = giocatori.drop('RIM_O', axis = 1)
giocatori = giocatori.drop('T1_PER', axis = 1)
giocatori = giocatori.drop('T2_PER', axis = 1)
giocatori = giocatori.drop('T3_PER', axis = 1)

players_without_role = giocatori.loc[(giocatori["ruolo"].isnull()) | (giocatori["ruolo"] == "NaN")]
players_without_role = players_without_role.drop('ruolo', axis = 1)

giocatori = giocatori.dropna()

players_without_role = scale_cols(players_without_role, 'minuti')
players_without_role = players_without_role.drop('minuti', axis = 1)
players_without_role["altezza"] = players_without_role["altezza"].replace(0, np.nan, inplace=True)

giocatori = scale_cols(giocatori, 'minuti')
giocatori = giocatori.drop('minuti', axis = 1)

x = giocatori.drop('ruolo', axis=1)
y = giocatori['ruolo']

# Effettua la codifica one-hot, rende y un vettore [0,0,0,0]
le = LabelEncoder()
y = le.fit_transform(y)
Y = to_categorical(y)

def create_model(neurons_first_layer, neurons_second_layer):
    model = Sequential()
    model.add(Dense(neurons_first_layer, activation='relu', input_shape=(15,)))
    model.add(Dense(neurons_second_layer, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=1000, batch_size=32, verbose=0)

neurons_first_layer = [32, 64, 128]
neurons_second_layer = [16, 32, 64]
param_grid = dict(neurons_first_layer=neurons_first_layer, neurons_second_layer=neurons_second_layer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x, Y)


# Stampa dei risultati
print("Migliori parametri: ", grid_result.best_params_)
print("Miglior punteggio: ", grid_result.best_score_)

imputer = SimpleImputer(strategy='mean')
prova = imputer.fit_transform(players_without_role)
y_pred = grid.predict(players_without_role)
def predizione(pred):
    strings = ["Ala", "Play", "Centro", "Guardia"]
    return [strings[np.argmax(vector)] for vector in pred]

print(predizione(y_pred))

C:\Users\feder\Desktop\Progetto-ICON-mappazone\classificatori\Utility.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, float_cols] = df[float_cols].mul(factor, axis=0)
C:\Users\feder\Desktop\Progetto-ICON-mappazone\classificatori\Utility.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, float_cols] = df[float_cols].mul(factor, axis=0)
C:\Users\feder\AppData\Local\Temp\ipykernel_20216\239889814.py:53: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.

MOTIVO DELL'ERRORE:
Il modello di imputazione (in questo caso SimpleImputer) ha bisogno di valori numeri validi per poter funzionare.
Se si desidera mantenere i dati None e non assegnare loro un valore, è possibile utilizzare una soluzione alternativa, ad esempio rimuovendo le righe che contengono dati mancanti o utilizzando un altro metodo di imputazione.
Tuttavia, questo potrebbe avere un impatto sul modello e sulla sua capacità di fare previsioni accurate.